In [54]:
import pandas as pd
import plotly.graph_objs as go 
import matplotlib.pyplot as plt
from scripts.xgboost_model import XGBRegressor
from tqdm import tqdm 
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Import des données

In [55]:
apple = pd.read_csv("../data/AAPL.csv")
amazon = pd.read_csv("../data/AMZN.csv")
berkshire = pd.read_csv("../data/BRK-B.csv")
google = pd.read_csv("../data/GOOG.csv")
meta = pd.read_csv("../data/META.csv")
microsoft = pd.read_csv("../data/MSFT.csv")
nvidia = pd.read_csv("../data/NVDA.csv")
tesla = pd.read_csv("../data/TSLA.csv")
unitedhealth = pd.read_csv("../data/UNH.csv")
exxon = pd.read_csv("../data/XOM.csv")

In [56]:
titres = {"Apple" : apple, 
          "Amazon" : amazon,
          "Berkshire" : berkshire,
          "Google" : google,
          "Meta" : meta,
          "Microsoft" : microsoft,
          "Nvidia" : nvidia,
          "Tesla" : tesla,
          "UnitedHealth" : unitedhealth,
          "Exxon" : exxon}

In [57]:
apple.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

## test de FE

# Calcul des prédictions

In [58]:
def stock_forecast(stock):

    stock['Date'] = pd.to_datetime(stock['Date'])
    stock = stock[stock["Date"]>="2020-07-01"]
    stock['Adj Close'] = stock['Adj Close'].pct_change()
    stock['High'] = stock['High'].pct_change()
    stock['Low'] = stock['Low'].pct_change()
    stock['Volume'] = stock['Volume'].pct_change()
    stock['dailyChange'] = (stock['Adj Close'] - stock['Open']) / stock['Open']

    stock['VolMA10'] = stock['Volume'].rolling(window=10).mean()

    stock['priceDirection'] = (stock['Adj Close'].shift(-1) - stock['Adj Close'])

    stock['Std_dev']=  stock['Adj Close'].rolling(10).std()

    stock['Williams%R'] = (stock['High'].max() - stock['Adj Close'])/(stock['High'] - stock['Low'].min()) * -100
    stock = stock.replace([np.inf, -np.inf], np.nan)
    stock['Volatility_HighLow_10days'] = stock['High'].rolling(window=10).std() / stock['Low'].rolling(window=10).mean()

    stock['Return'] = stock['Close'].pct_change()
    stock = stock.dropna()
    stock = stock.drop(columns=['Adj Close'])

    stock['Target'] = stock['Return'].shift(-10)
    stock = stock.dropna()

    window_size = 30
    step_size = 7  
    prediction_days = 7
    
    prediction_dates = []
    predictions = []
  
    features_to_drop = ['Date', 'Target', 'Volume']
    for start in tqdm(range(0, len(stock) - window_size - prediction_days, step_size)):

        end = start + window_size
        train = stock.iloc[start:end]
        test = stock.iloc[end:end + prediction_days]
    
        model = XGBRegressor(n_estimators=400, max_depth=30, learning_rate=0.01, verbosity=0)

        model.fit(train.drop(features_to_drop, axis=1), train['Target'])

    
        pred = model.predict(test.drop(['Date', 'Target', 'Volume'], axis=1))
        predictions.append(pred)
    
        prediction_date = stock.iloc[end + prediction_days - 1]['Date']
        prediction_dates.append(prediction_date)

    predictions = [value[6] for value in predictions]


    df_predictions = pd.DataFrame({
        'Date': prediction_dates,
        'Prediction': predictions
    })
    
    merged_df = stock.merge(df_predictions, on='Date', how='inner')
    
    merged_df = merged_df.drop(columns = ["Target"])
    
    return merged_df

In [59]:
predictions_dic = {}

In [60]:
for key in titres :
    predictions_dic[key] = stock_forecast(titres[key])

/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_48097/3643416133.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_48097/3643416133.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/cs/hh3_26_57sx7l8sdk574zbdh0000gn/T/ipykernel_48097/3643416133.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [61]:
dfs_to_concat = []

for name, df in predictions_dic.items():

    df['Date'] = pd.to_datetime(df['Date'])
    print(df.columns)
    dfs_to_concat.append(df[['Date', 'Prediction', "Return"]])

portfolio_predictions = pd.concat(dfs_to_concat)

portfolio_avg_predictions = portfolio_predictions.groupby('Date').mean().reset_index()

print(portfolio_avg_predictions.head())

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'dailyChange',
       'VolMA10', 'priceDirection', 'Std_dev', 'Williams%R',
       'Volatility_HighLow_10days', 'Return', 'Prediction'],
      dtype='object')
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'dailyChange',
       'VolMA10', 'priceDirection', 'Std_dev', 'Williams%R',
       'Volatility_HighLow_10days', 'Return', 'Prediction'],
      dtype='object')
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'dailyChange',
       'VolMA10', 'priceDirection', 'Std_dev', 'Williams%R',
       'Volatility_HighLow_10days', 'Return', 'Prediction'],
      dtype='object')
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'dailyChange',
       'VolMA10', 'priceDirection', 'Std_dev', 'Williams%R',
       'Volatility_HighLow_10days', 'Return', 'Prediction'],
      dtype='object')
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'dailyChange',
       'VolMA10', 'priceDirection', 'Std_dev', 'Williams%R',
   

In [62]:
portfolio_avg_predictions['Real_Portfolio_Value'] = 1000 * (1 + portfolio_avg_predictions['Return']).cumprod()
portfolio_avg_predictions['Predicted_Portfolio_Value'] = 1000 * (1 + portfolio_avg_predictions['Prediction']).cumprod()

In [63]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = portfolio_avg_predictions.Date, 
                         y = portfolio_avg_predictions['Predicted_Portfolio_Value'],
                         mode = 'lines',
                         name ='Return_predict', 
                         fill = "tozeroy"))

fig.add_trace(go.Scatter(x = portfolio_avg_predictions.Date, 
                         y = portfolio_avg_predictions["Real_Portfolio_Value"],
                         mode = 'lines',
                         name ='Return_reel', 
                         fill = "tozeroy"))

fig.update_layout(title="Évolution de la valeur de ptf",
                  xaxis_title='Date',
                  yaxis_title='Valeur')

fig.show()